In [16]:
# input:1 2 3 4 5 6 7 8 9 10  
# output: 1 27 125 343 729
%%writefile tcpservere.py
import socket
host=socket.gethostname()
port=5000
server_socket=socket.socket()
server_socket.bind((host,port))
server_socket.listen()
conn,address=server_socket.accept()
data=conn.recv(1024).decode()
print('from connected user:',data)
conn.close()

Overwriting tcpservere.py


In [17]:
%%writefile tcpcliente.py
import socket
host=socket.gethostname()
# host='406-1-16'
port=5000
client_socket=socket.socket()
client_socket.connect((host,port))
message=input('->')
l=message.split()
n=[]
for i in l:
    n.append(int(i))
odd=list(filter(lambda x:x%2!=0,n))
cube=list(map(lambda x:x**3,odd))
ans=''
for i in cube:
    ans=ans+str(i)+" "
client_socket.send(ans.encode())
client_socket.close()

Overwriting tcpcliente.py


In [21]:
#send file as an input and recive file and create file as output
%%writefile server.py
import socket
host='localhost'
port=65432
Buffer_size=1024
with socket.socket() as s:
    s.bind((host,port))
    s.listen()
    conn,addr=s.accept()
    with conn:
        print('connect by',addr)
        file_data=b""
        while True:
            data=conn.recv(Buffer_size)
            if not data:
                break
            file_data=data
        with open('recv.ipynb','wb') as f:
            f.write(file_data)
        print('file recived')

Overwriting server.py


In [22]:
%%writefile client.py
import socket

host = "localhost"
port = 65432
Buffer_size = 1024
with socket.socket() as s:
    s.connect((host,port))
    with open('chapter 7 7.1.ipynb','rb') as f:
        while True:
            data = f.read(Buffer_size)
            if not data:
                break
            s.sendall(data)
        print('file send')

Overwriting client.py


In [71]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score

with open('vishal_new.html','r',encoding='utf-8') as file:
    html_content=file.read()
soup=BeautifulSoup(html_content)
# print(soup.prettify())
box=soup.findAll('div',class_='_2kHMtA')
name=[]
rating=[]
sprice=[]
price=[]
for i in box:
    name.append(i.find('div',class_='_4rR01T').get_text())
    rating.append(float(i.find('div',class_='_3LWZlK').get_text()))
    sprice.append(int(i.find('div',class_='_30jeq3 _1_WHN1').text[1:3]+
                  i.find('div',class_='_30jeq3 _1_WHN1').text[4:]))
    try:
        price.append(int(i.find('div',class_='_3I9_wc _27UcVY').text[1:].replace(",","")))
    except:
        price.append(np.nan)

d={'name':name,'rating':rating,'sprice':sprice,'price':price}
df=pd.DataFrame(d)
df
df.dropna(inplace=True)
x=df[['rating','sprice']]
y=df['price']  
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

model=LinearRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

print(mean_squared_error(y_test,y_pred))
print(r2_score(y_test,y_pred))

226610141.93806237
0.11614205640107322
